In [89]:

# coding: utf-8

# In[ ]:


from __future__ import print_function

import sys
import numpy as np

from time import time
import matplotlib.pyplot as plt 

sys.path.append('/home/xilinx')
from pynq import Overlay
from pynq import allocate

if __name__ == "__main__":
    print("Entry:", sys.argv[0])
    print("System argument(s):", len(sys.argv))

    print("Start of \"" + sys.argv[0] + "\"")

    ol = Overlay("./design_1.bit")
    ipMATMUL = ol.matrixmul_0

    

    inBuffer0 = allocate(shape=(3,3), dtype=np.int32)
    inBuffer1 = allocate(shape=(3,3), dtype=np.int32)
    outBuffer0 = allocate(shape=(3,3), dtype=np.int32)

    
    a = np.random.randint(3,  size=(3, 3))
    b = np.random.randint(5,  size=(3, 3))
    
    print('a:')
    print(a)
    print("============================")
    print('b:')
    print(b)
    print("============================\n\n")
    
    for i in range(3):
        for j in range(3):
            inBuffer0[i][j] = a[i][j]
            inBuffer1[i][j] = b[i][j]
    
    timeKernelStart = time()
    for i in range(3):
        for j in range(3):
            ipMATMUL.write(64+4*(i*4+j), int(inBuffer0[i][j]))
            ipMATMUL.write(128+4*(i*4+j), int(inBuffer1[i][j]))
            

    ipMATMUL.write(0x00, 0x01)
    while (ipMATMUL.read(0x00) & 0x4) == 0x0:
        continue

    timeKernelEnd = time()
    print("Kernel execution time: " + str(timeKernelEnd - timeKernelStart) + " s\n\n")
    for i in range(3):
        for j in range(3):
            outBuffer0[i][j] = ipMATMUL.read(192+4*(i*3+j))

    
    ANS = np.array(a) @ np.array(b) 
    print('ANS:')
    print(ANS)
    print("============================")
    print('result:')
    print(outBuffer0)
    print("============================")
    if np.allclose(ANS, np.array(outBuffer0)):
        print('PASS')
    else:
        print('WRONG')
    
    print("============================")
    print("Exit process")



Entry: /usr/lib/python3/dist-packages/ipykernel_launcher.py
System argument(s): 3
Start of "/usr/lib/python3/dist-packages/ipykernel_launcher.py"
a:
[[2 1 2]
 [0 0 0]
 [2 0 0]]
b:
[[0 0 4]
 [3 1 0]
 [0 0 2]]


Kernel execution time: 0.003975629806518555 s


ANS:
[[ 3  1 12]
 [ 0  0  0]
 [ 0  0  8]]
result:
[[ 3  1 12]
 [ 0  0  0]
 [ 0  0  8]]
PASS
Exit process
